<a href="https://colab.research.google.com/github/pasupuletikishore/Customer-Retention-Intelligence-Engine/blob/main/Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Cell 1: Install necessary libraries
!pip install -q streamlit pandas numpy scikit-learn joblib
print("✅ Libraries installed successfully.")

✅ Libraries installed successfully.


In [2]:
import pandas as pd
import numpy as np

# 1. Define the columns and random data
rows = 100  # Number of customers to simulate
data = {
    'customerID': [f'{i+1000}-TEST' for i in range(rows)],
    'gender': np.random.choice(['Male', 'Female'], rows),
    'SeniorCitizen': np.random.choice([0, 1], rows),
    'Partner': np.random.choice(['Yes', 'No'], rows),
    'Dependents': np.random.choice(['Yes', 'No'], rows),
    'tenure': np.random.randint(1, 73, rows),
    'PhoneService': np.random.choice(['Yes', 'No'], rows),
    'MultipleLines': np.random.choice(['No phone service', 'No', 'Yes'], rows),
    'InternetService': np.random.choice(['DSL', 'Fiber optic', 'No'], rows),
    'OnlineSecurity': np.random.choice(['Yes', 'No'], rows),
    'OnlineBackup': np.random.choice(['Yes', 'No'], rows),
    'DeviceProtection': np.random.choice(['Yes', 'No'], rows),
    'TechSupport': np.random.choice(['Yes', 'No'], rows),
    'StreamingTV': np.random.choice(['Yes', 'No'], rows),
    'StreamingMovies': np.random.choice(['Yes', 'No'], rows),
    'Contract': np.random.choice(['Month-to-month', 'One year', 'Two year'], rows),
    'PaperlessBilling': np.random.choice(['Yes', 'No'], rows),
    'PaymentMethod': np.random.choice(['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)'], rows),
    'MonthlyCharges': np.random.uniform(18.25, 118.75, rows).round(2),
    'TotalCharges': np.random.uniform(18.8, 8000.0, rows).round(2),
    'Churn': np.random.choice(['Yes', 'No'], rows)
}

# 2. Create DataFrame
df = pd.DataFrame(data)

# 3. Save as CSV
file_name = 'wa_fn-usec_-telco-customer-churn.csv'
df.to_csv(file_name, index=False)

print(f"✅ Success! File '{file_name}' has been created with {rows} rows.")
print("👉 You can now use this file in the 'Batch Processing' tab of your App!")

✅ Success! File 'wa_fn-usec_-telco-customer-churn.csv' has been created with 100 rows.
👉 You can now use this file in the 'Batch Processing' tab of your App!


In [6]:
# Cell 2: Upload the Dataset
import os
from google.colab import files
import pandas as pd
import numpy as np

# --- OPTION A: UPLOAD REAL DATA ---
print("⬇️ Please upload 'wa_fn-usec_-telco-customer-churn.csv' below:")
uploaded = files.upload()

# Rename file to standard name if uploaded with a different name
for filename in uploaded.keys():
    if "csv" in filename:
        os.rename(filename, 'wa_fn-usec_-telco-customer-churn.csv')
        print("✅ File uploaded and renamed to 'wa_fn-usec_-telco-customer-churn.csv'")

# --- OPTION B: UNCOMMENT BELOW IF YOU DONT HAVE THE FILE ---
# (Only use this if you cannot download from Kaggle)
# def create_dummy_data():
#     rows = 1000
#     data = {
#         'customerID': range(rows),
#         'gender': np.random.choice(['Male', 'Female'], rows),
#         'SeniorCitizen': np.random.choice([0, 1], rows),
#         'Partner': np.random.choice(['Yes', 'No'], rows),
#         'Dependents': np.random.choice(['Yes', 'No'], rows),
#         'tenure': np.random.randint(1, 73, rows),
#         'PhoneService': np.random.choice(['Yes', 'No'], rows),
#         'MultipleLines': np.random.choice(['No phone service', 'No', 'Yes'], rows),
#         'InternetService': np.random.choice(['DSL', 'Fiber optic', 'No'], rows),
#         'OnlineSecurity': np.random.choice(['Yes', 'No'], rows),
#         'OnlineBackup': np.random.choice(['Yes', 'No'], rows),
#         'DeviceProtection': np.random.choice(['Yes', 'No'], rows),
#         'TechSupport': np.random.choice(['Yes', 'No'], rows),
#         'StreamingTV': np.random.choice(['Yes', 'No'], rows),
#         'StreamingMovies': np.random.choice(['Yes', 'No'], rows),
#         'Contract': np.random.choice(['Month-to-month', 'One year', 'Two year'], rows),
#         'PaperlessBilling': np.random.choice(['Yes', 'No'], rows),
#         'PaymentMethod': np.random.choice(['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)'], rows),
#         'MonthlyCharges': np.random.uniform(18, 118, rows).round(2),
#         'TotalCharges': np.random.uniform(18, 8000, rows).round(2),
#         'Churn': np.random.choice(['Yes', 'No'], rows)
#     }
#     df = pd.DataFrame(data)
#     df.to_csv('wa_fn-usec_-telco-customer-churn.csv', index=False)
#     print("⚠️ Synthetic data created for testing.")

# create_dummy_data() # <--- Un-comment this line only if you didn't upload a file

⬇️ Please upload 'wa_fn-usec_-telco-customer-churn.csv' below:


Saving wa_fn-usec_-telco-customer-churn.csv to wa_fn-usec_-telco-customer-churn (1).csv
✅ File uploaded and renamed to 'wa_fn-usec_-telco-customer-churn.csv'


In [3]:
%%writefile train_model.py
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import joblib

# Constants
DATA_PATH = 'wa_fn-usec_-telco-customer-churn.csv'
MODEL_PATH = 'churn_model.pkl'

def train_model():
    try:
        df = pd.read_csv(DATA_PATH)
        print(f"✅ Data loaded. Shape: {df.shape}")
    except FileNotFoundError:
        print("❌ Data file not found.")
        return

    # Basic Cleaning
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce').fillna(0)
    df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

    # Capitalize headers to ensure matching
    df.columns = [c[0].upper() + c[1:] for c in df.columns]

    # Features
    features = ['Tenure', 'PhoneService', 'Contract', 'PaperlessBilling',
                'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'InternetService']

    X = df[features]
    y = df['Churn']

    # Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Preprocessing
    cat_cols = ['PhoneService', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'InternetService']
    num_cols = ['Tenure', 'MonthlyCharges', 'TotalCharges']

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_cols),
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
        ])

    # Pipeline
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(n_estimators=50, random_state=42))
    ])

    # Train
    print("⏳ Training Model...")
    pipeline.fit(X_train, y_train)

    # Save
    joblib.dump(pipeline, MODEL_PATH)
    acc = accuracy_score(y_test, pipeline.predict(X_test))
    print(f"🎉 Model saved to {MODEL_PATH}. Accuracy: {acc:.2f}")

if __name__ == "__main__":
    train_model()

Writing train_model.py


In [4]:
# Cell 4: Run the training script
!python train_model.py

✅ Data loaded. Shape: (100, 21)
⏳ Training Model...
🎉 Model saved to churn_model.pkl. Accuracy: 0.55


In [5]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
import numpy as np

# --- 1. CONFIGURATION & SETUP ---
st.set_page_config(
    page_title="Customer Retention Intelligence",
    page_icon="🛡️",
    layout="wide"
)

# Custom CSS to make it look professional
st.markdown("""
    <style>
    .stMetric {
        background-color: #f0f2f6;
        padding: 10px;
        border-radius: 10px;
    }
    </style>
    """, unsafe_allow_html=True)

# --- 2. LOAD TRAINED MODEL ---
@st.cache_resource
def load_model():
    try:
        return joblib.load('churn_model.pkl')
    except FileNotFoundError:
        return None

pipeline = load_model()

# --- 3. HEADER & LOGIC ---
st.title("🛡️ Customer Retention Intelligence Engine")
st.markdown("Use AI to predict which customers are leaving and generate **Automated Retention Plans**.")

if pipeline is None:
    st.error("🚨 Model not found! Please run the training script (train_model.py) first.")
    st.stop()

# --- 4. TABS SYSTEM (The Pro Feature) ---
tab1, tab2 = st.tabs(["👤 Single Customer (Simulator)", "📂 Bulk Analysis (Enterprise Mode)"])

# ==========================================
# TAB 1: SINGLE CUSTOMER SIMULATOR
# ==========================================
with tab1:
    st.markdown("### 📝 adjust profile to test scenarios")
    col1, col2, col3 = st.columns([1, 1, 1])

    with col1:
        st.subheader("1. Service Details")
        tenure = st.slider('Tenure (Months)', 0, 72, 12, help="How long they have been a customer.")
        contract = st.selectbox('Contract Type', ['Month-to-month', 'One year', 'Two year'])
        internet = st.selectbox('Internet Service', ['DSL', 'Fiber optic', 'No'])

    with col2:
        st.subheader("2. Financials")
        monthly = st.slider('Monthly Charges ($)', 0.0, 150.0, 70.0)
        payment = st.selectbox('Payment Method', ['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)'])
        # Auto-calculate total based on tenure (Approx)
        total_charges = tenure * monthly

    with col3:
        st.subheader("3. Settings")
        phone = st.selectbox('Phone Service', ['Yes', 'No'])
        paperless = st.selectbox('Paperless Billing', ['Yes', 'No'])

    # Create Dataframe for Prediction
    input_df = pd.DataFrame({
        'Tenure': [tenure], 'PhoneService': [phone], 'Contract': [contract],
        'PaperlessBilling': [paperless], 'PaymentMethod': [payment],
        'MonthlyCharges': [monthly], 'TotalCharges': [total_charges], 'InternetService': [internet]
    })

    st.markdown("---")

    # Prediction Button
    if st.button("🚀 Run Prediction Analysis", type="primary"):
        # Get Probability
        prob = pipeline.predict_proba(input_df)[0][1]

        # Display Results
        c1, c2 = st.columns(2)

        with c1:
            st.subheader("Risk Assessment")
            st.metric("Churn Probability", f"{prob:.1%}")

            if prob > 0.5:
                st.error("⚠️ STATUS: HIGH RISK CUSTOMER")
            else:
                st.success("✅ STATUS: LOYAL CUSTOMER")

        with c2:
            st.subheader("💡 Recommended Retention Strategy")
            if prob > 0.5:
                st.warning("Action Required! The system suggests:")
                if contract == 'Month-to-month':
                    st.write("👉 **Offer:** Upgrade to 1-Year Contract with 15% Discount.")
                if internet == 'Fiber optic':
                    st.write("👉 **Support:** Schedule proactive router checkup (Fiber latency detected).")
                if monthly > 80:
                    st.write("👉 **Perk:** Add free streaming bundle to increase value.")
            else:
                st.info("No retention budget needed.")
                st.write("📧 Send standard 'Thank You' email.")

# ==========================================
# TAB 2: BATCH PROCESSING (ENTERPRISE)
# ==========================================
with tab2:
    st.header("📂 Bulk Data Processor")
    st.info("Upload a CSV file to predict churn for thousands of customers at once.")

    uploaded_file = st.file_uploader("Upload 'wa_fn-usec_-telco-customer-churn.csv'", type=["csv"])

    if uploaded_file is not None:
        try:
            df = pd.read_csv(uploaded_file)
            # Take a sample of 200 rows if file is huge, just for speed in demo
            if len(df) > 500:
                df = df.sample(500)
                st.caption("⚡ Processing random sample of 500 rows for speed...")

            # Clean & Prep
            if 'TotalCharges' in df.columns:
                df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce').fillna(0)

            # Define features expected by model
            features = ['Tenure', 'PhoneService', 'Contract', 'PaperlessBilling',
                        'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'InternetService']

            # Check if columns exist
            if all(col in df.columns for col in features):
                # Predict
                probs = pipeline.predict_proba(df[features])[:, 1]

                # Append Results
                df['Churn_Probability'] = probs
                df['Risk_Level'] = ['High Risk ⚠️' if p > 0.5 else 'Safe ✅' for p in probs]

                # Logic for Strategy
                conditions = [
                    (df['Churn_Probability'] > 0.5) & (df['Contract'] == 'Month-to-month'),
                    (df['Churn_Probability'] > 0.5) & (df['InternetService'] == 'Fiber optic'),
                    (df['Churn_Probability'] > 0.5)
                ]
                choices = ['Convert to Annual', 'Tech Support Call', 'General Discount']
                df['Action_Plan'] = np.select(conditions, choices, default='No Action')

                # --- DASHBOARD ---
                st.divider()
                m1, m2, m3 = st.columns(3)
                m1.metric("Customers Analyzed", len(df))
                m2.metric("High Risk Identified", len(df[df['Risk_Level'] == 'High Risk ⚠️']), delta_color="inverse")
                m3.metric("Potential Revenue at Risk", f"${df[df['Risk_Level'] == 'High Risk ⚠️']['MonthlyCharges'].sum():,.2f}")

                st.subheader("📋 Prioritized Action List")
                st.dataframe(df[['customerID', 'Risk_Level', 'Churn_Probability', 'Action_Plan']].sort_values(by='Churn_Probability', ascending=False).head(10))

                # Download
                csv = df.to_csv(index=False).encode('utf-8')
                st.download_button("📥 Download Full Report", csv, "churn_report.csv", "text/csv")

            else:
                st.error(f"CSV is missing columns! Needed: {features}")

        except Exception as e:
            st.error(f"Error: {e}")

Writing app.py


In [6]:
# Cell 6: Run App with Ngrok (High Speed)
!pip install -q pyngrok

from pyngrok import ngrok
import time

# 1. Kill any old processes so it starts fresh
!pkill streamlit
ngrok.kill()

# 2. Authenticate with YOUR token
ngrok.set_auth_token("37slGatQW5XJPXCnZK9zgVebK76_6AT7gyz1gSexM4kKT3iBf")

# 3. Start Streamlit in the background
print("🚀 Starting Streamlit...")
get_ipython().system_raw('streamlit run app.py &')

# Wait 3 seconds for the server to start
time.sleep(3)

# 4. Open the secure tunnel
try:
    public_url = ngrok.connect(8501).public_url
    print(f"✅ SUCCESS! CLICK THIS LINK TO SEE YOUR PROJECT: {public_url}")
except Exception as e:
    print(f"❌ Error: {e}")

🚀 Starting Streamlit...
✅ SUCCESS! CLICK THIS LINK TO SEE YOUR PROJECT: https://unpolluting-bubbly-kaylie.ngrok-free.dev
